In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas --upgrade
!pip install transformers
!pip install torch
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
!pip install scipy --upgrade
import ipdb
%pdb off
!pip install tensorflow --upgrade

!pip install language_tool_python

In [ ]:

import pandas as pd
import nltk
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import torch

import preprocessor as p

import pycountry
import re
import string
from nltk import tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os



### topic modelling
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

#hate
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
device = torch.device("cuda")

In [ ]:
def load_topic_model(user_feeds):
        global topic_classes

        MODEL = f"cardiffnlp/tweet-topic-21-multi"

        with torch.no_grad():
            topic_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

        topic_classes = topic_model.config.id2label#

        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        tokens = user_feeds.clean_text.apply(lambda row: tokenizer(row, return_tensors='pt'))

        print('loaded topic model')
        print(tokens)
        return topic_model , tokens

In [ ]:
def load_psysentimento_model(user_feeds):
        
        tweets = user_feeds.clean_text.to_list()

        # hateful
        analyzer = create_analyzer(task="hate_speech", lang="en")
        hate_labels = ['hateful', 'targeted', 'aggressive']

        hate_out = [ analyzer.predict(preprocess_tweet(txt)) for txt in tweets ]
        print('predicted hate of tweets')

        
        print('loaded hate model')

        #emotion
        e_analyzer = create_analyzer(task="emotion", lang="en")
        emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear']     


        emo_out = [ e_analyzer.predict(preprocess_tweet(txt)) for txt in tweets ]
        print('predicted emotion of tweets')

        print('loaded emotion model')

        return hate_out, emo_out

In [ ]:
import csv

In [ ]:
def tweet_cleaner(tw_list):
        print("yes")
        remove_rt = lambda x: re.sub('RT @\w+: '," ",str(x))
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(x))
        hash = lambda x: re.sub(r'#', "", str(x))
        amp = lambda x: re.sub(r'&amp', "", str(x))


        print(tw_list[:10])
        tw_list['grammartext'] = tw_list['text'].map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)

        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        return tw_list

In [ ]:
def get_sentiment(row):
        score = SentimentIntensityAnalyzer().polarity_scores(row)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if neg > pos:
            
            label = "negative"
        elif pos > neg:
            
            label = "positive"
        else:
            
            label = "neutral"
        return [label, neg,neu,pos, comp]



def get_topic(topic_model, row):
        output = topic_model(**row.to(device))
        scores = output[0][0].detach().cpu().numpy()
        scores = expit(scores)
        pred = np.argmax(scores)
        return [pred] + scores.flatten().tolist()


def get_hate(row):
        hate_labels = ['hateful', 'targeted', 'aggressive']
        return [row.probas[hate_labels[i]] for i in range(3) ]

def get_emo(row):
        emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear'] 
        return [row.probas[emo_labels[i]] for i in range(7) ]


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/historic_tweets/get_useful/history_tweets_retweeter_FINAL_FINAL_v3.csv')

user_feeds = tweet_cleaner(df)
topic_model, topic_in = load_topic_model(user_feeds)
hate, emo = load_psysentimento_model(user_feeds)


sent = user_feeds.copy().clean_text.to_list()
grammer_in = user_feeds.grammartext.tolist()

sent = user_feeds.copy().clean_text.to_list()
grammer_in = user_feeds.grammartext.tolist()


sent_output = [ get_sentiment(row) for row in sent]


topic_output = [ get_topic(topic_model, row) for row in topic_in]


hate_output = [ get_hate(row) for row in hate]


hate_output = abs(np.array(hate_output) - np.mean(hate_output, axis=0)).tolist() 
hate_output = abs(np.array(hate_output) - np.mean(hate_output, axis=0)).tolist()

emo_output = [ get_emo(row) for row in emo]


all = np.hstack((sent_output,topic_output,hate_output,emo_output))

topics = [ val for _,val in topic_classes.items()]

hate_labels = ['hateful', 'targeted', 'aggressive']
emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear']     

cols = ['sentiment','neg','neu','pos','comp','topic'] + topics + hate_labels + emo_labels



user_feeds_df = pd.DataFrame(all.tolist())
user_feeds_df.columns = cols


user_feeds_df['id'] = list(user_feeds.index.values)
user_feeds_df['id'] = list(user_feeds['id'].tolist())

int_cols = ['neg','neu','pos','comp'] + topics + hate_labels + emo_labels

user_feeds_df[int_cols] = user_feeds_df.copy()[int_cols].astype(float)

user_feeds = user_feeds_df.copy()

del user_feeds_df # delete the old user_feeds_df

fid = list(set(user_feeds['id'].tolist()))

df = df.copy()[df['id'].isin(fid)]

user_ids = df['id'].tolist()

hates = ['hateful', 'targeted', 'aggressive']
emos = ['joy','sadness','others','anger','surprise','disgust','fear']     

del topic_classes, hate_labels, emo_labels

score_cols = ['id','neg','neu','pos','comp']+topics + hates + emos 

score_df = user_feeds[score_cols]

label_df = user_feeds[['id','sentiment','topic']]

def Average(lst):
    return sum(lst) / len(lst)
    
import random
from collections import Counter
from itertools import groupby
def cust_mode(l):
    freqs = groupby(Counter(l).most_common(), lambda x:x[1])
    return [val for val,count in next(freqs)[1]]

all_s = []
all_mode = []
all_count = []


for user in user_ids:

    s_df = score_df[score_df['id']==user].drop('id',axis=1).values.tolist()
    
    l_df = label_df[label_df['id']==user].drop('id',axis=1).values.tolist()

    all_s.append([ Average(x) for x in zip(*s_df) ])

    all_mode.append( [ random.choice(cust_mode(x)) for x in zip(*l_df)] )

    counts = [ x for x in zip(*l_df) ] 

    sent_count = [ counts[0].count(s) for s in ['negative','neutral','positive'] ]
    topic_count = [ counts[1].count(str(float(s))) for s in range(19) ]

    all_count.append(sent_count + topic_count)


new_cols = [ f'user_{x}_mean' for x in ['neg','neu','pos','comp']+topics + hates + emos ]# + ['politeness'] ]

df[new_cols]  = all_s

new_col = [ f'user_{x}_mode' for x in ['sentiment','topic']]

df[new_col] = all_mode

df.reset_index(drop=False)
df.set_index('id', inplace = True)


df.to_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_historic_features/1/historic_retweeter_features_msc_FINAL_NEW_v3.csv')

In [ ]:
df.columns

In [ ]:

if 1435553820382269441 in df.id.values:
    print("yes")

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_historic_features/1/historic_retweeter_features_msc_FINAL_NEW_v3.csv')